#Instaling essential libraries

In [1]:
!pip install hazm

#Importing libraries

In [2]:
import hazm
import nltk
import pandas as pd

#English Dataset

In [3]:
!wget http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xf MovieSummaries.tar.gz

--2023-11-10 17:33:08--  http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48002242 (46M) [application/x-gzip]
Saving to: ‘MovieSummaries.tar.gz.6’

MovieSummaries.tar. 100%[===================>]  45.78M  1.60MB/s    in 29s     

2023-11-10 17:33:37 (1.60 MB/s) - ‘MovieSummaries.tar.gz.6’ saved [48002242/48002242]



In [4]:
english_df = pd.read_csv("/content/MovieSummaries/plot_summaries.txt", delimiter = "\t",names=["id","text"])
english_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42303 entries, 0 to 42302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      42303 non-null  int64 
 1   text    42303 non-null  object
dtypes: int64(1), object(1)
memory usage: 661.1+ KB


In [5]:
english_df.head()

,id,text
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [6]:
english_df.iloc[333:343]

,id,text
333,3182458,The stories follow a black formal tailcoat as ...
334,14901197,"Crossing the plains, a wagon train comes acros..."
335,474750,A new heroine has arrived in Gotham whose iden...
336,32777283,"Justin Bayard, a Northern Territory policeman,..."
337,6002416,The central character is the Post Master who i...
338,3322205,Pooja Dharamchand is the daughter of a rich M...
339,3006994,The film features two anonymous Scottish-accen...
340,9537791,"Sam, a college student in a small Northwestern..."
341,23910715,"She's in for a wild ride, when Jean Madison , ..."
342,45772,"On November 25, 1975, Rocky Balboa is introd..."


**import essential libraries**

In [7]:
import string
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pickle
import os
import re

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

first we look for null values in english dataset

In [8]:
print(english_df.isnull().sum())

id      0
text    0
dtype: int64


**step 1: tokenizing words**

then we tokenize the `text` column by words

In [9]:
def tokenize_text(df, column_name, tokenized_column_name):
    df[column_name] = df[column_name].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    df[tokenized_column_name] = df[column_name].apply(lambda x: word_tokenize(x))
    return df

processed_df = tokenize_text(english_df, 'text', 'tokenized_text')

In [10]:
english_df.loc[0, 'tokenized_text']

['Shlykov',
 'a',
 'hardworking',
 'taxi',
 'driver',
 'and',
 'Lyosha',
 'a',
 'saxophonist',
 'develop',
 'a',
 'bizarre',
 'lovehate',
 'relationship',
 'and',
 'despite',
 'their',
 'prejudices',
 'realize',
 'they',
 'arent',
 'so',
 'different',
 'after',
 'all']

**step 2: normalizing tokens**

after tokenizing, we normalize the `tokenized_text` column. This includes lowecasing the tokens.

In [11]:
def normalize_text(text):
    normalized_tokens = [token.lower() for token in text]
    return normalized_tokens

english_df['normalized_tokens'] = english_df['tokenized_text'].apply(normalize_text)

In [12]:
english_df.loc[0, 'normalized_tokens']

['shlykov',
 'a',
 'hardworking',
 'taxi',
 'driver',
 'and',
 'lyosha',
 'a',
 'saxophonist',
 'develop',
 'a',
 'bizarre',
 'lovehate',
 'relationship',
 'and',
 'despite',
 'their',
 'prejudices',
 'realize',
 'they',
 'arent',
 'so',
 'different',
 'after',
 'all']

**step 3: stemming normalized tokens**

In [13]:
def stem_normalized_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

english_df['stemmed_tokens'] = english_df['normalized_tokens'].apply(stem_normalized_tokens)

In [14]:
english_df.loc[0, 'stemmed_tokens']

['shlykov',
 'a',
 'hardwork',
 'taxi',
 'driver',
 'and',
 'lyosha',
 'a',
 'saxophonist',
 'develop',
 'a',
 'bizarr',
 'loveh',
 'relationship',
 'and',
 'despit',
 'their',
 'prejudic',
 'realiz',
 'they',
 'arent',
 'so',
 'differ',
 'after',
 'all']

**step 4: removing stopwords**

In [15]:
stop_words = set(stopwords.words('english'))

def remove_stop_words(tokens):
    return [token for token in tokens if token not in stop_words and token.isalpha()]

english_df['final_tokens'] = english_df['stemmed_tokens'].apply(remove_stop_words)

In [16]:
english_df.loc[0, 'final_tokens']

['shlykov',
 'hardwork',
 'taxi',
 'driver',
 'lyosha',
 'saxophonist',
 'develop',
 'bizarr',
 'loveh',
 'relationship',
 'despit',
 'prejudic',
 'realiz',
 'arent',
 'differ']

**building the inverted index**

In [17]:
def build_inverted_index(df, column_name):
    inverted_index = {}

    for index, row in df.head(1000).iterrows():
        document_id = index
        stemmed_tokens = row[column_name]

        term_frequency = {}
        for term in stemmed_tokens:
            term_frequency[term] = term_frequency.get(term, 0) + 1

        for term, frequency in term_frequency.items():
            if term in inverted_index:
                inverted_index[term].append((frequency, document_id))
                inverted_index[term] = sorted(inverted_index[term], key=lambda x: x[0], reverse=True)

            else:
                inverted_index[term] = [(frequency, document_id)]

    return inverted_index

In [18]:
import time

start = time.time()
english_inverted_index = build_inverted_index(english_df, 'final_tokens')
end = time.time()

print("total execution time for english: ", end-start)

# here's an example of how it looks
english_inverted_index['psychologist']

total execution time for english:  2.397524356842041


[(2, 738), (1, 6), (1, 662), (1, 808), (1, 866), (1, 878), (1, 983)]

In [19]:
def save_invertedindex_to_file(file_name, inverted_index):
    with open(f'{file_name}.pkl', 'wb') as file:
        pickle.dump(inverted_index, file)

    file_size = os.path.getsize(f'{file_name}.pkl')
    print(f'Inverted index saved to file. File size: {file_size} bytes')

In [20]:
save_invertedindex_to_file('English_Inverted_Index', english_inverted_index)

Inverted index saved to file. File size: 1018813 bytes


**building the term occurence matrix (binary coded)**

In [21]:
def build_term_occurences(inverted_index):
    term_occurrences = {}
    unique_doc_ids = []

    for postings in inverted_index.values():
        for _, doc_id in postings:
            if doc_id not in unique_doc_ids:
                unique_doc_ids.append(doc_id)

    for term, postings in inverted_index.items():
        term_occurrence_list = [1 if doc_id in [doc_id for _, doc_id in postings] else 0 for doc_id in unique_doc_ids]
        term_occurrences[term] = term_occurrence_list

    return term_occurrences


In [22]:
english_term_occurences = build_term_occurences(english_inverted_index)

**boolean IR**

In [24]:
import re

def query_processing(term_occurrences):
    query = input("please enter your query: ")

    substrings = re.split(r'([+.])', query)

    terms = []
    terms_bin = []
    expressions = []
    binary_code = ""

    for sub in substrings:
        # terms
        if sub and sub not in ['+', '.']:
            if sub.startswith('!'):
                if sub[1:] not in term_occurrences:
                    binary_code = [0] * 1000
                else:
                    binary_code = term_occurrences[sub[1:]]
                    binary_code = [1 if bit == 0 else 0 for bit in binary_code]
                terms.append(sub[1:])
            else:
                if sub not in term_occurrences:
                    binary_code = [0] * 1000
                else:
                    binary_code = term_occurrences[sub]
                terms.append(sub)

            terms_bin.append(binary_code)

        # expressions
        elif sub:
            expressions.append(sub)

    for i, exp in enumerate(expressions):
        if exp == '+':
            result_or = [bit1 | bit2 for bit1, bit2 in zip(terms_bin[i], terms_bin[i+1])]
            terms_bin[i+1] = result_or
        elif exp == '.':
            result_and = [bit1 & bit2 for bit1, bit2 in zip(terms_bin[i], terms_bin[i+1])]
            terms_bin[i+1] = result_and

    result = terms_bin[-1]
    documents = [index  for index, value in enumerate(result) if value == 1]

    print("These documents are found: ", documents)


In [25]:
query_processing(english_term_occurences)

please enter your query: complex+!city
These documents are found:  [90, 122, 184, 207, 264, 268, 297, 304, 318, 536, 847, 951]


In [26]:
def count_tokens_and_documents(df, column_name):
    total_tokens = sum(df[column_name].apply(len))
    total_documents = len(df)
    return total_tokens, total_documents

In [27]:
total_english_tokens, total_english_documents = count_tokens_and_documents(english_df, 'final_tokens')

print(f'Total tokens: {total_english_tokens}')
print(f'Total documents: {total_english_documents}')

Total tokens: 7489549
Total documents: 42303


#Persian Dataset

In [28]:
!wget https://github.com/mohamad-dehghani/persian-pdf-books-dataset/raw/master/final_books.xlsx

--2023-11-10 17:42:24--  https://github.com/mohamad-dehghani/persian-pdf-books-dataset/raw/master/final_books.xlsx
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohamad-dehghani/persian-pdf-books-dataset/master/final_books.xlsx [following]
--2023-11-10 17:42:24--  https://raw.githubusercontent.com/mohamad-dehghani/persian-pdf-books-dataset/master/final_books.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1380625 (1.3M) [application/octet-stream]
Saving to: ‘final_books.xlsx.4’

final_books.xlsx.4  100%[===================>]   1.32M  --.-KB/s    in 0.05s   

2023-11-10 17:42:25 (27.0 

In [29]:
farsi_df = pd.read_excel('/content/final_books.xlsx')
farsi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     2824 non-null   object
 1   date      2824 non-null   object
 2   content   2441 non-null   object
 3   category  2824 non-null   object
 4   author    2824 non-null   object
 5   comments  2824 non-null   object
dtypes: object(6)
memory usage: 132.5+ KB


In [30]:
farsi_df.head()

,title,date,content,category,author,comments
0,بهترین درسهای زندگی‎,۲ دی ۱۳۹۸,تی دی جیکس یک کشیش، نویسنده، سخنران آمریکایی ...,روانشناسی,سارا رزولت,بدون دیدگاه
1,از سلاجقه تا صفویه,۱ دی ۱۳۹۸,کتاب حاضر که اینک در دسترس خوانندگان محترم قر...,تاریخ ایران,نصرت الله مشکوتی,بدون دیدگاه
2,سرگذشت رضاشاه,۲۹ آذر ۱۳۹۸,دکتر فتح الله بینا تحصیلات خود را در رشته پزش...,تاریخ ایران,فتح الله بینا,بدون دیدگاه
3,سیری نوین در فرگشت و اسرار مغز,۲۸ آذر ۱۳۹۸,مجموعه پیش رو مطالبیست که در گروه Evolution ت...,علم و دانش,مانی منوچهری,بدون دیدگاه
4,آسمان پرستاره,۲۶ آذر ۱۳۹۸,کتاب آسمان پرستاره نوشته‌ی کتی هایدن، به زبان...,کودکان و نوجوانان,کتی هایدن,بدون دیدگاه


In [31]:
farsi_df.iloc[440:480]

,title,date,content,category,author,comments
440,صوتی هاشمی بدون روتوش,۱۸ بهمن ۱۳۹۶,کتاب صوتی «هاشمی بدون روتوش» مجموعه گفت‌و‌گوها...,دسته‌بندی نشده,صادق زیباکلام، فرشته‌السادات اتفاق‌فر,۱ دیدگاه
441,تاریخ زندگی اقتصادی روستاییان و طبقات اجتماعی...,۱۷ بهمن ۱۳۹۶,موضوع این کتاب چنانکه از نام آن برمی آید دربا...,تاریخ ایران,غلامرضا انصافپور,۱ دیدگاه
442,کاهش وزن سریع با ۳۰ تکنیک طلایی,۱۵ بهمن ۱۳۹۶,با کتاب کاهش وزن سریع با ۳۰ تکنیک طلایی نوشته...,پزشکی و سلامت,ابراهیم رجب زاده,۲ دیدگاه
443,صوتی روانشناسی زنان,۱۵ بهمن ۱۳۹۶,کتاب صوتی «روانشناسی زنان» (Feminine Psycholog...,روانشناسی,کارن هورنای,بدون دیدگاه
444,ایران در عهد باستان,۱۳ بهمن ۱۳۹۶,ایران در عهد باستان: در تاریخ اقوام و پادشاها...,تاریخ ایران,محمدجواد مشکور,بدون دیدگاه
445,صوتی ناکامی چپ در ایران : شورشیان آرمانخواه,۱۲ بهمن ۱۳۹۶,کتاب صوتی «ناکامی چپ در ایران : شورشیان آرمان...,دسته‌بندی نشده,مازیار بهروز,بدون دیدگاه
446,آن زندگی را انتخاب کنید که می خواهید,۱۰ بهمن ۱۳۹۶,اواسط ژانویه بود. در حیاط دانشگاه هاروارد به ...,روانشناسی,تال بن شاهار,بدون دیدگاه
447,درباره‌ی زمان,۹ بهمن ۱۳۹۶,مفهوم زمان چندان آشنا و نزدیک است، که نامفهوم...,فلسفه,شروین وکیلی,بدون دیدگاه
448,صوتی شهریار,۸ بهمن ۱۳۹۶,کتاب صوتی «شهریار» اثر نیکولو ماکیاولی (۱۵۲۷-۱...,دسته‌بندی نشده,نیکولو ماکیاولی,۲ دیدگاه
449,PHP به زبان ساده,۸ بهمن ۱۳۹۶,در این کتاب سعی شده است که مطالب به زبان و کده...,برنامه نویسی,یونس ابراهیمی,بدون دیدگاه


**import essential libraries**

In [32]:
from hazm import word_tokenize
from hazm import Normalizer
from hazm import Stemmer
from hazm import stopwords_list
import re

first we look for null values in english dataset

In [33]:
print(farsi_df.isnull().sum())

title         0
date          0
content     383
category      0
author        0
comments      0
dtype: int64


looks like 383 columns of `content` is missing. There are 2 options: we can delete the rows or just leave them blank. The first option is not efficeint since we are removing so much information. In next steps, we will us titles instead of content.

**remove ponctuations**

In [34]:
def remove_punctuation(text):
    if pd.notna(text):
        return re.sub(r'[^\w\s]|[.،؛]', '', text)
    else:
        return text

farsi_df['title'] = farsi_df['title'].apply(remove_punctuation)
farsi_df['content'] = farsi_df['content'].apply(remove_punctuation)
farsi_df['category'] = farsi_df['category'].apply(remove_punctuation)
farsi_df['author'] = farsi_df['author'].apply(remove_punctuation)


In [35]:
farsi_df['all_info'] = (
    farsi_df['title'].fillna('') +
    ' ' +
    farsi_df['content'].fillna('') +
    ' ' +
    farsi_df['category'].fillna('') +
    ' ' +
    farsi_df['author'].fillna('')
)

In [36]:
farsi_df.loc[0, 'all_info']

' بهترین درسهای زندگی  تی دی جیکس یک کشیش نویسنده سخنران آمریکایی است سمینار های تی دی جیکس اغلب انگیزشی و مذهبی است به همین دلیل او در ایران به سخنران انگیزشی سیاه پوست لقب گرفته است در زندگینامه تی دی جیکس شاهد فعالیتهای مختلفی هستیم که هر کدام از آنها خواهان تجربه و تلاش زیادی هستند مواردی که اعلام شد تنها بخشی از فعالیتهای جیکس است او هم اکنون کشیش خانه پاتر است و مخاطبان بسیاری دارد موعظههای\xa0 و سخنرانیها تی دی جیکس در کلیسا از شبکههای تلویزیونی و رادیویی مختلف پخش میشوند این کتاب گزیده ای از سخنرانی های آقای جیکز  یکی از بهترین سخنران های انگیزشی در دنیا است امید است با مطالعه دقیق و باور عمیق تغییری مثبت در زندگیتان ایجاد شود روانشناسی           سارا رزولت  '

**step 1: tokenizing words**

then we tokenize the `text` column by words

In [38]:
farsi_df['tokens'] = farsi_df['all_info'].apply(lambda x: word_tokenize(x))

In [39]:
farsi_df.loc[0, 'tokens']

['بهترین',
 'درسهای',
 'زندگی',
 'تی',
 'دی',
 'جیکس',
 'یک',
 'کشیش',
 'نویسنده',
 'سخنران',
 'آمریکایی',
 'است',
 'سمینار',
 'های',
 'تی',
 'دی',
 'جیکس',
 'اغلب',
 'انگیزشی',
 'و',
 'مذهبی',
 'است',
 'به',
 'همین',
 'دلیل',
 'او',
 'در',
 'ایران',
 'به',
 'سخنران',
 'انگیزشی',
 'سیاه',
 'پوست',
 'لقب',
 'گرفته_است',
 'در',
 'زندگینامه',
 'تی',
 'دی',
 'جیکس',
 'شاهد',
 'فعالیتهای',
 'مختلفی',
 'هستیم',
 'که',
 'هر',
 'کدام',
 'از',
 'آنها',
 'خواهان',
 'تجربه',
 'و',
 'تلاش',
 'زیادی',
 'هستند',
 'مواردی',
 'که',
 'اعلام',
 'شد',
 'تنها',
 'بخشی',
 'از',
 'فعالیتهای',
 'جیکس',
 'است',
 'او',
 'هم',
 'اکنون',
 'کشیش',
 'خانه',
 'پاتر',
 'است',
 'و',
 'مخاطبان',
 'بسیاری',
 'دارد',
 'موعظههای\xa0',
 'و',
 'سخنرانیها',
 'تی',
 'دی',
 'جیکس',
 'در',
 'کلیسا',
 'از',
 'شبکههای',
 'تلویزیونی',
 'و',
 'رادیویی',
 'مختلف',
 'پخش',
 'میشوند',
 'این',
 'کتاب',
 'گزیده_ای',
 'از',
 'سخنرانی',
 'های',
 'آقای',
 'جیکز',
 'یکی',
 'از',
 'بهترین',
 'سخنران',
 'های',
 'انگیزشی',
 'در',
 'دنیا',
 'ا

**step 2: normalizing tokens**

after tokenizing, we normalize the `tokenized_text` column. This includes lowecasing the tokens.

In [40]:
normalizer = Normalizer()
normalize_tokens = lambda tokens: [normalizer.normalize(token) for token in tokens]
farsi_df['normalized_tokens'] = farsi_df['tokens'].apply(normalize_tokens)

In [41]:
farsi_df.loc[0, 'normalized_tokens']

['بهترین',
 'درسهای',
 'زندگی',
 'تی',
 'دی',
 'جیکس',
 'یک',
 'کشیش',
 'نویسنده',
 'سخنران',
 'آمریکایی',
 'است',
 'سمینار',
 'های',
 'تی',
 'دی',
 'جیکس',
 'اغلب',
 'انگیزشی',
 'و',
 'مذهبی',
 'است',
 'به',
 'همین',
 'دلیل',
 'او',
 'در',
 'ایران',
 'به',
 'سخنران',
 'انگیزشی',
 'سیاه',
 'پوست',
 'لقب',
 'گرفته_است',
 'در',
 'زندگینامه',
 'تی',
 'دی',
 'جیکس',
 'شاهد',
 'فعالیتهای',
 'مختلفی',
 'هستیم',
 'که',
 'هر',
 'کدام',
 'از',
 'آنها',
 'خواهان',
 'تجربه',
 'و',
 'تلاش',
 'زیادی',
 'هستند',
 'مواردی',
 'که',
 'اعلام',
 'شد',
 'تنها',
 'بخشی',
 'از',
 'فعالیتهای',
 'جیکس',
 'است',
 'او',
 'هم',
 'اکنون',
 'کشیش',
 'خانه',
 'پاتر',
 'است',
 'و',
 'مخاطبان',
 'بسیاری',
 'دارد',
 'موعظه\u200cهای',
 'و',
 'سخنرانیها',
 'تی',
 'دی',
 'جیکس',
 'در',
 'کلیسا',
 'از',
 'شبکه\u200cهای',
 'تلویزیونی',
 'و',
 'رادیویی',
 'مختلف',
 'پخش',
 'می\u200cشوند',
 'این',
 'کتاب',
 'گزیده_ای',
 'از',
 'سخنرانی',
 'های',
 'آقای',
 'جیکز',
 'یکی',
 'از',
 'بهترین',
 'سخنران',
 'های',
 'انگیزشی',
 'در'

**step 3: stemming normalized tokens**

In [42]:
stemmer = Stemmer()
farsi_df['stemmed_tokens'] = farsi_df['normalized_tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

In [43]:
farsi_df.loc[0, 'stemmed_tokens']

['به',
 'درس',
 'زندگ',
 'ت',
 'د',
 'جیکس',
 'یک',
 'کش',
 'نویسنده',
 'سخنر',
 'آمریکا',
 'اس',
 'سمینار',
 '',
 'ت',
 'د',
 'جیکس',
 'اغلب',
 'انگیزش',
 'و',
 'مذهب',
 'اس',
 'به',
 'همین',
 'دلیل',
 'او',
 'در',
 'ایر',
 'به',
 'سخنر',
 'انگیزش',
 'سیاه',
 'پوس',
 'لقب',
 'گرفته_اس',
 'در',
 'زندگینامه',
 'ت',
 'د',
 'جیکس',
 'شاهد',
 'فعالیت',
 'مختلف',
 'هست',
 'که',
 'هر',
 'کدا',
 'از',
 'آن',
 'خواه',
 'تجربه',
 'و',
 'تلا',
 'زیاد',
 'هستند',
 'موارد',
 'که',
 'اعلا',
 'شد',
 'تن',
 'بخش',
 'از',
 'فعالیت',
 'جیکس',
 'اس',
 'او',
 'ه',
 'اکنون',
 'کش',
 'خانه',
 'پا',
 'اس',
 'و',
 'مخاطب',
 'بسیار',
 'دارد',
 'موعظه',
 'و',
 'سخنرانی',
 'ت',
 'د',
 'جیکس',
 'در',
 'کلیسا',
 'از',
 'شبکه',
 'تلویزیون',
 'و',
 'رادیو',
 'مختلف',
 'پخ',
 'می\u200cشوند',
 'این',
 'کتاب',
 'گزیده_ا',
 'از',
 'سخنران',
 '',
 'آقا',
 'جیکز',
 'یک',
 'از',
 'به',
 'سخنر',
 '',
 'انگیزش',
 'در',
 'دنیا',
 'اس',
 'امید',
 'اس',
 'با',
 'مطالعه',
 'دقیق',
 'و',
 'باور',
 'عمیق',
 'تغییر',
 'مثب',
 'در'

**step 4: removing stopwords**

In [44]:
persian_stopwords = set(stopwords_list())
custom_stopwords = ['از', 'به', 'با', 'در', 'برای', 'بر', 'آن', 'او', 'این', 'ها', 'بله', 'آره', 'نه', 'اون', 'که', 'بدون', 'هر', 'شاید', 'باید', 'خیلی']

farsi_df['final_tokens'] = farsi_df['stemmed_tokens'].apply(lambda tokens: [token for token in tokens if token not in persian_stopwords])
farsi_df['final_tokens'] = farsi_df['final_tokens'].apply(lambda tokens: [token for token in tokens if token not in custom_stopwords])

In [45]:
farsi_df.loc[0, 'final_tokens']

['درس',
 'زندگ',
 'ت',
 'د',
 'جیکس',
 'کش',
 'نویسنده',
 'سخنر',
 'آمریکا',
 'اس',
 'سمینار',
 '',
 'ت',
 'د',
 'جیکس',
 'انگیزش',
 'مذهب',
 'اس',
 'دلیل',
 'ایر',
 'سخنر',
 'انگیزش',
 'سیاه',
 'پوس',
 'لقب',
 'گرفته_اس',
 'زندگینامه',
 'ت',
 'د',
 'جیکس',
 'شاهد',
 'فعالیت',
 'مختلف',
 'هست',
 'کدا',
 'خواه',
 'تجربه',
 'تلا',
 'موارد',
 'اعلا',
 'تن',
 'فعالیت',
 'جیکس',
 'اس',
 'ه',
 'کش',
 'خانه',
 'پا',
 'اس',
 'مخاطب',
 'موعظه',
 'سخنرانی',
 'ت',
 'د',
 'جیکس',
 'کلیسا',
 'شبکه',
 'تلویزیون',
 'رادیو',
 'مختلف',
 'پخ',
 'کتاب',
 'گزیده_ا',
 'سخنران',
 '',
 'آقا',
 'جیکز',
 'سخنر',
 '',
 'انگیزش',
 'دنیا',
 'اس',
 'امید',
 'اس',
 'مطالعه',
 'دقیق',
 'باور',
 'عمیق',
 'مثب',
 'زندگ',
 'ایجاد',
 'روانشناس',
 'سارا',
 'رزول']

removing tokens that have 1 or 0 characters

In [46]:
farsi_df['final_tokens'] = farsi_df['final_tokens'].apply(lambda tokens: [token for token in tokens if token and len(token) > 1])

In [47]:
farsi_df.loc[0, 'final_tokens']

['درس',
 'زندگ',
 'جیکس',
 'کش',
 'نویسنده',
 'سخنر',
 'آمریکا',
 'اس',
 'سمینار',
 'جیکس',
 'انگیزش',
 'مذهب',
 'اس',
 'دلیل',
 'ایر',
 'سخنر',
 'انگیزش',
 'سیاه',
 'پوس',
 'لقب',
 'گرفته_اس',
 'زندگینامه',
 'جیکس',
 'شاهد',
 'فعالیت',
 'مختلف',
 'هست',
 'کدا',
 'خواه',
 'تجربه',
 'تلا',
 'موارد',
 'اعلا',
 'تن',
 'فعالیت',
 'جیکس',
 'اس',
 'کش',
 'خانه',
 'پا',
 'اس',
 'مخاطب',
 'موعظه',
 'سخنرانی',
 'جیکس',
 'کلیسا',
 'شبکه',
 'تلویزیون',
 'رادیو',
 'مختلف',
 'پخ',
 'کتاب',
 'گزیده_ا',
 'سخنران',
 'آقا',
 'جیکز',
 'سخنر',
 'انگیزش',
 'دنیا',
 'اس',
 'امید',
 'اس',
 'مطالعه',
 'دقیق',
 'باور',
 'عمیق',
 'مثب',
 'زندگ',
 'ایجاد',
 'روانشناس',
 'سارا',
 'رزول']

**building the inverted index**

In [48]:
import time

start = time.time()
farsi_inverted_index = build_inverted_index(farsi_df, 'final_tokens')
end = time.time()

print("The total execution time for Farsi: ", end-start)

The total execution time for Farsi:  1.9613368511199951


**building the term occurence matrix (binary coded)**

In [49]:
farsi_term_occurrences = build_term_occurences(farsi_inverted_index)

**boolean IR**

In [50]:
query_processing(farsi_term_occurrences)

please enter your query: کتاب+روانشناس+!نمایش
These documents are found:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 

In [51]:
total_farsi_tokens, total_farsi_documents = count_tokens_and_documents(farsi_df, 'final_tokens')

print(f'Total tokens: {total_farsi_tokens}')
print(f'Total documents: {total_farsi_documents}')

Total tokens: 217858
Total documents: 2824


In [52]:
save_invertedindex_to_file('Farsi_Inverted_Index', farsi_inverted_index)

Inverted index saved to file. File size: 731068 bytes
